In [ ]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidi

In [ ]:
!pip install pycocotools
!pip install transformers torchvision timm --quiet
!pip install wandb


In [ ]:
!pip install -q gdown
!gdown --id 1pJ3xfKtHiTdysX5G3dxqKTdGESOBYCxJ --output "/content/drive/MyDrive/DI725/Asg2/AU-AIR.zip"
!unzip -q "/content/drive/MyDrive/DI725/Asg2/AU-AIR.zip" -d "/content/drive/MyDrive/DI725/Asg2/AU-AIR"


# Preprocessing

In [ ]:
import json

with open('/content/drive/MyDrive/DI725/Asg2/annotations.json') as f:
    data = json.load(f)

### Clean the keys of annotations from ':'.

In [ ]:
import json

# File paths
input_path = "/content/drive/MyDrive/DI725/Asg2/annotations.json"
output_path = "/content/drive/MyDrive/DI725/Asg2/annotations.json"

# Recursive function to clean dictionary keys
def clean_keys(obj):
    if isinstance(obj, dict):
        return {k.rstrip(":"): clean_keys(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [clean_keys(i) for i in obj]
    else:
        return obj

# Load original data
with open(input_path, "r") as f:
    raw_data = json.load(f)

# Clean all keys
cleaned_data = clean_keys(raw_data)

# Save cleaned data
with open(output_path, "w") as f:
    json.dump(cleaned_data, f, indent=4)

print(f"✅ Cleaned annotations saved to: {output_path}")


✅ Cleaned annotations saved to: /content/drive/MyDrive/DI725/Asg2/annotations.json


### Convert the annotations to COCO format

In [ ]:
import json
import os

# Step 1: Load cleaned annotation file
with open("/content/drive/MyDrive/DI725/Asg2/annotations.json", "r") as f:
    auair_data = json.load(f)

# Step 2: Convert to COCO format
def convert_auair_to_coco(auair_data, output_path="auair_coco.json"):
    images = []
    annotations = []

    # Define COCO-style categories from class names
    categories = [{"id": i, "name": name} for i, name in enumerate(auair_data["categories"])]

    ann_id = 1  # unique annotation ID
    for img_id, entry in enumerate(auair_data["annotations"]):
        image_name = entry["image_name"]
        width = entry["image_width"]
        height = entry["image_height"]

        images.append({
            "id": img_id,
            "file_name": image_name,
            "width": width,
            "height": height
        })

        for bbox in entry["bbox"]:
            x = bbox["left"]
            y = bbox["top"]
            w = bbox["width"]
            h = bbox["height"]
            category_id = bbox["class"]

            annotations.append({
                "id": ann_id,
                "image_id": img_id,
                "category_id": category_id,
                "bbox": [x, y, w, h],       # COCO: [x, y, width, height]
                "area": w * h,
                "iscrowd": 0
            })
            ann_id += 1

    coco_data = {
        "images": images,
        "annotations": annotations,
        "categories": categories
    }

    # Step 3: Save to COCO format JSON file
    with open(output_path, "w") as f:
        json.dump(coco_data, f, indent=4)

    print(f"✅ COCO-format annotations saved to: {output_path}")

# Step 4: Run the conversion
convert_auair_to_coco(
    auair_data,
    output_path="/content/drive/MyDrive/DI725/Asg2/auair_coco.json"
)


✅ COCO-format annotations saved to: /content/drive/MyDrive/DI725/Asg2/auair_coco.json


### Filter the COCO annotations such that images that are not annotated and/or annotated but no image file exists

In [ ]:
import os
import json

# === PATHS ===
image_folder = "/content/drive/MyDrive/DI725/Asg2/AU-AIR/images"
annotation_file = "/content/drive/MyDrive/DI725/Asg2/auair_coco.json"
filtered_annotation_path = "/content/drive/MyDrive/DI725/Asg2/auair_coco_filtered.json"

# === LOAD ANNOTATION ===
with open(annotation_file, "r") as f:
    coco = json.load(f)

# === IMAGE NAME CHECKS ===
annotated_image_names = set(img["file_name"] for img in coco["images"])
image_files = set(os.listdir(image_folder))

missing_images = annotated_image_names - image_files
unannotated_images = image_files - annotated_image_names

# === REPORT ===
print(f"✅ Total annotated images: {len(annotated_image_names)}")
print(f"📂 Total actual image files: {len(image_files)}\n")

print(f"❌ Missing image files (in annotations but not found on disk): {len(missing_images)}")
if missing_images:
    print("Examples:", sorted(list(missing_images))[:5])

print(f"📝 Unannotated image files (exist in folder but not in annotations): {len(unannotated_images)}")
if unannotated_images:
    print("Examples:", sorted(list(unannotated_images))[:5])

# === FILTER ANNOTATIONS ===
valid_images = [img for img in coco["images"] if img["file_name"] in image_files]
valid_image_ids = set(img["id"] for img in valid_images)
valid_annotations = [ann for ann in coco["annotations"] if ann["image_id"] in valid_image_ids]

filtered_coco = {
    "images": valid_images,
    "annotations": valid_annotations,
    "categories": coco["categories"]
}

# === SAVE CLEANED JSON ===
with open(filtered_annotation_path, "w") as f:
    json.dump(filtered_coco, f, indent=4)

print(f"\n✅ Filtered COCO annotations saved to: {filtered_annotation_path}")
print(f"➡️ Valid images remaining: {len(valid_images)}")
print(f"➡️ Valid annotations remaining: {len(valid_annotations)}")


✅ Total annotated images: 32823
📂 Total actual image files: 26895

❌ Missing image files (in annotations but not found on disk): 5928
Examples: ['frame_20190906150731_x_0001193.jpg', 'frame_20190906150731_x_0001194.jpg', 'frame_20190906150731_x_0001195.jpg', 'frame_20190906150731_x_0001196.jpg', 'frame_20190906150731_x_0001197.jpg']
📝 Unannotated image files (exist in folder but not in annotations): 0

✅ Filtered COCO annotations saved to: /content/drive/MyDrive/DI725/Asg2/auair_coco_filtered.json
➡️ Valid images remaining: 26895
➡️ Valid annotations remaining: 101619


In [ ]:
from pycocotools.coco import COCO
from PIL import Image
from torch.utils.data import Dataset
import torch
import os

class AUAirHFDataset(Dataset):
    def __init__(self, image_dir, annotation_path, processor):
        self.coco = COCO(annotation_path)
        self.image_dir = image_dir
        self.ids = list(self.coco.imgs.keys())
        self.processor = processor

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]
        file_path = os.path.join(self.image_dir, img_info['file_name'])
        image = Image.open(file_path).convert("RGB")

        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)

        # Add required fields to annotations
        target = {
            "image_id": img_id,
            "annotations": [
                {
                    "bbox": ann["bbox"],
                    "category_id": ann["category_id"],
                    "area": ann["bbox"][2] * ann["bbox"][3],  # width * height
                    "iscrowd": ann.get("iscrowd", 0)
                }
                for ann in anns
            ]
        }

        # Process image and annotations
        encoding = self.processor(images=image, annotations=target, return_tensors="pt")

        # Squeeze tensor outputs (pixel_values, pixel_mask), keep labels as is
        encoding = {
            k: (v.squeeze(0) if isinstance(v, torch.Tensor) else v)
            for k, v in encoding.items()
        }

        return encoding

    def __len__(self):
        return len(self.ids)


In [ ]:
import json
import random

# Load COCO JSON
with open("/content/drive/MyDrive/DI725/Asg2/auair_coco_filtered.json", "r") as f:
    coco_data = json.load(f)

# Shuffle images
random.seed(42)
all_images = coco_data["images"]
random.shuffle(all_images)

# Split image metadata
test_images = all_images[-2823:]
trainval_images = all_images[:-2823]

# Further split train/val (e.g., 90/10)
val_split = int(0.1 * len(trainval_images))
val_images = trainval_images[:val_split]
train_images = trainval_images[val_split:]

# Helper: get image ids for splitting annotations
def get_annos(image_list):
    ids = set(img["id"] for img in image_list)
    return [a for a in coco_data["annotations"] if a["image_id"] in ids]

# Prepare separate COCO-style dicts
def create_subset(images, annotations, categories):
    return {"images": images, "annotations": annotations, "categories": categories}

train_json = create_subset(train_images, get_annos(train_images), coco_data["categories"])
val_json = create_subset(val_images, get_annos(val_images), coco_data["categories"])
test_json = create_subset(test_images, get_annos(test_images), coco_data["categories"])

# Save the new splits
with open("/content/drive/MyDrive/DI725/Asg2/auair_train.json", "w") as f:
    json.dump(train_json, f, indent=4)

with open("/content/drive/MyDrive/DI725/Asg2/auair_val.json", "w") as f:
    json.dump(val_json, f, indent=4)

with open("/content/drive/MyDrive/DI725/Asg2/auair_test.json", "w") as f:
    json.dump(test_json, f, indent=4)

print("✅ Train/Val/Test splits saved.\n")

# === Print stats ===
total = len(all_images)
train_count = len(train_images)
val_count = len(val_images)
test_count = len(test_images)

train_pct = 100 * train_count / total
val_pct = 100 * val_count / total
test_pct = 100 * test_count / total

print(f"📊 Total images: {total}")
print(f"🟢 Train: {train_count} images ({train_pct:.2f}%)")
print(f"🔵 Val: {val_count} images ({val_pct:.2f}%)")
print(f"🟠 Test: {test_count} images ({test_pct:.2f}%)")

# Optional: annotation count for extra info
print(f"\n📝 Total annotations in filtered dataset: {len(coco_data['annotations'])}")


✅ Train/Val/Test splits saved.

📊 Total images: 26895
🟢 Train: 21665 images (80.55%)
🔵 Val: 2407 images (8.95%)
🟠 Test: 2823 images (10.50%)

📝 Total annotations in filtered dataset: 101619


In [ ]:
from transformers import DetrImageProcessor
from torch.utils.data import DataLoader


# Use the filtered and split files
train_ann_path = "/content/drive/MyDrive/DI725/Asg2/auair_train.json"
val_ann_path = "/content/drive/MyDrive/DI725/Asg2/auair_val.json"


processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

train_dataset = AUAirHFDataset(image_folder, train_ann_path, processor)
val_dataset = AUAirHFDataset(image_folder, val_ann_path, processor)


train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: x)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn=lambda x: x)


loading annotations into memory...
Done (t=0.90s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


# Modeling

In [ ]:
from transformers import DetrConfig, DetrImageProcessor, DetrForObjectDetection
import torch

# 1. Define config with correct number of classes
num_classes = 9  # 8 AU-AIR categories + background
config = DetrConfig.from_pretrained("facebook/detr-resnet-50", num_labels=num_classes)

# 2. Load processor and model (ignore classifier size mismatch)
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained(
    "facebook/detr-resnet-50",
    config=config,
    ignore_mismatched_sizes=True
)

# 3. Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("✅ DETR model loaded with config (9 classes) and pretrained backbone")


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DetrForObjectDetection were not initialized from the model checkpoin

✅ DETR model loaded with config (9 classes) and pretrained backbone


In [30]:
import wandb

sweep_config = {
    "method": "grid",
    "name": "detr_auair_sweep",
    "metric": {
        "name": "val_loss",
        "goal": "minimize"
    },
    "parameters": {
        "learning_rate": {"values": [1e-5, 5e-5, 1e-4]},
        "weight_decay": {"values": [0.0, 1e-4]},
        "batch_size": {"values": [2, 4]}
    }
}

sweep_id = wandb.sweep(sweep_config, project="DI725_Asg2_HyperParamTuning")


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x79f80538f090>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

Create sweep with ID: 6toqohht
Sweep URL: https://wandb.ai/aliyigitbasaran-/DI725_Asg2_HyperParamTuning/sweeps/6toqohht
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x79f80538f090>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
def compute_iou(box1, box2):
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = box1_area + box2_area - inter_area
    return inter_area / union if union != 0 else 0

def calculate_map(pred_boxes, pred_labels, true_boxes, true_labels, iou_threshold=0.5):
    matched = 0
    for pb, pl in zip(pred_boxes, pred_labels):
        for tb, tl in zip(true_boxes, true_labels):
            if pl == tl and compute_iou(pb, tb) >= iou_threshold:
                matched += 1
                break
    return matched / len(true_labels) if len(true_labels) > 0 else 0.0


In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json
import torch
from torch.utils.data import DataLoader
import wandb

def wandb_train():
    wandb.init(project="DI725_Asg2_HyperParamTuning")
    config = wandb.config

    global model
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)

    # Dataset and loaders
    train_dataset = AUAirHFDataset(image_folder, train_ann_path, processor)
    val_dataset = AUAirHFDataset(image_folder, val_ann_path, processor)
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, collate_fn=lambda x: x)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, collate_fn=lambda x: x)

    log_interval = 50  # log to wandb every 100 batches

    for epoch in range(5):
        model.train()
        total_loss = 0

        for step, batch in enumerate(train_loader):
            pixel_values = torch.stack([item["pixel_values"] for item in batch]).to(device)
            pixel_mask = torch.stack([item["pixel_mask"] for item in batch]).to(device)
            labels = [{k: v.to(device) for k, v in item["labels"][0].items()} for item in batch]

            outputs = model(pixel_values=pixel_values, pixel_mask=pixel_mask, labels=labels)
            loss = outputs.loss

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_loss += loss.item()

            # Log train loss every log_interval steps
            if (step + 1) % log_interval == 0:
                wandb.log({
                    "step": step + 1 + epoch * len(train_loader),
                    "running_train_loss": total_loss / (step + 1)
                })

        # === Evaluation ===
        model.eval()
        val_loss = 0
        coco_preds = []

        with torch.no_grad():
            for batch in val_loader:
                pixel_values = torch.stack([item["pixel_values"] for item in batch]).to(device)
                pixel_mask = torch.stack([item["pixel_mask"] for item in batch]).to(device)
                labels = [{k: v.to(device) for k, v in item["labels"][0].items()} for item in batch]

                outputs = model(pixel_values=pixel_values, pixel_mask=pixel_mask, labels=labels)
                val_loss += outputs.loss.item()

                logits = outputs.logits.softmax(-1).cpu()
                boxes = outputs.pred_boxes.cpu()

                for i in range(len(batch)):
                    keep = logits[i].max(-1).values > 0.7
                    scores = logits[i][keep].max(-1).values
                    labels_pred = logits[i][keep].argmax(-1)
                    boxes_xywh = boxes[i][keep]

                    for box, label, score in zip(boxes_xywh, labels_pred, scores):
                        x1, y1, x2, y2 = box.tolist()
                        coco_preds.append({
                            "image_id": int(batch[i]["image_id"]),
                            "category_id": int(label),
                            "bbox": [x1, y1, x2 - x1, y2 - y1],
                            "score": float(score)
                        })

        val_loss /= len(val_loader)

        # === COCO Evaluation ===
        val_coco = COCO(val_ann_path)
        with open("tmp_detr_preds.json", "w") as f:
            json.dump(coco_preds, f)
        coco_dt = val_coco.loadRes("tmp_detr_preds.json")
        coco_eval = COCOeval(val_coco, coco_dt, "bbox")
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()

        # === Per-category AP calculation ===
        category_ap = {}
        for idx, cat_id in enumerate(val_coco.getCatIds()):
            name = val_coco.loadCats(cat_id)[0]["name"]
            ap = coco_eval.eval["precision"][0, :, idx, 0, 2].mean()
            category_ap[name] = float(ap) if not torch.isnan(torch.tensor(ap)) else 0.0

        mean_ap = sum(category_ap.values()) / len(category_ap)

        # === Log everything for the current epoch ===
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": total_loss / len(train_loader),
            "val_loss": val_loss,
            "val_mAP@0.5": mean_ap,
            **category_ap
        })


In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json
import torch
from torch.utils.data import DataLoader
import wandb

def wandb_train():
    wandb.init(project="DI725_Asg2_HyperParamTuning")
    config = wandb.config

    global model
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)

    # Dataset and loaders
    train_dataset = AUAirHFDataset(image_folder, train_ann_path, processor)
    val_dataset = AUAirHFDataset(image_folder, val_ann_path, processor)
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, collate_fn=lambda x: x)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, collate_fn=lambda x: x)

    log_interval = 50

    # Default val metrics to use in first log (will be updated after 1st epoch)
    prev_val_loss = 0.0
    prev_mean_ap = 0.0
    prev_category_ap = {}

    for epoch in range(5):
        model.train()
        total_loss = 0

        for step, batch in enumerate(train_loader):
            pixel_values = torch.stack([item["pixel_values"] for item in batch]).to(device)
            pixel_mask = torch.stack([item["pixel_mask"] for item in batch]).to(device)
            labels = [{k: v.to(device) for k, v in item["labels"][0].items()} for item in batch]

            outputs = model(pixel_values=pixel_values, pixel_mask=pixel_mask, labels=labels)
            loss = outputs.loss

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_loss += loss.item()

            if (step + 1) % log_interval == 0:
                # Log all metrics (some reused from previous epoch)
                wandb.log({
                    "step": step + 1 + epoch * len(train_loader),
                    "epoch": epoch + 1,
                    "running_train_loss": total_loss / (step + 1),
                    "learning_rate": optimizer.param_groups[0]["lr"],
                    "val_loss": prev_val_loss,
                    "val_mAP@0.5": prev_mean_ap,
                    **prev_category_ap
                })

        # === Evaluation ===
        model.eval()
        val_loss = 0
        coco_preds = []

        with torch.no_grad():
            for batch in val_loader:
                pixel_values = torch.stack([item["pixel_values"] for item in batch]).to(device)
                pixel_mask = torch.stack([item["pixel_mask"] for item in batch]).to(device)
                labels = [{k: v.to(device) for k, v in item["labels"][0].items()} for item in batch]

                outputs = model(pixel_values=pixel_values, pixel_mask=pixel_mask, labels=labels)
                val_loss += outputs.loss.item()

                logits = outputs.logits.softmax(-1).cpu()
                boxes = outputs.pred_boxes.cpu()

                for i in range(len(batch)):
                    keep = logits[i].max(-1).values > 0.7
                    scores = logits[i][keep].max(-1).values
                    labels_pred = logits[i][keep].argmax(-1)
                    boxes_xywh = boxes[i][keep]

                    for box, label, score in zip(boxes_xywh, labels_pred, scores):
                        x1, y1, x2, y2 = box.tolist()
                        coco_preds.append({
                            "image_id": int(batch[i]["image_id"]),
                            "category_id": int(label),
                            "bbox": [x1, y1, x2 - x1, y2 - y1],
                            "score": float(score)
                        })

        val_loss /= len(val_loader)

        # === COCO Evaluation ===
        val_coco = COCO(val_ann_path)
        with open("tmp_detr_preds.json", "w") as f:
            json.dump(coco_preds, f)
        coco_dt = val_coco.loadRes("tmp_detr_preds.json")
        coco_eval = COCOeval(val_coco, coco_dt, "bbox")
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()

        category_ap = {}
        for idx, cat_id in enumerate(val_coco.getCatIds()):
            name = val_coco.loadCats(cat_id)[0]["name"]
            ap = coco_eval.eval["precision"][0, :, idx, 0, 2].mean()
            category_ap[name] = float(ap) if not torch.isnan(torch.tensor(ap)) else 0.0

        mean_ap = sum(category_ap.values()) / len(category_ap)

        # Save for future batch-interval logging
        prev_val_loss = val_loss
        prev_mean_ap = mean_ap
        prev_category_ap = category_ap

        # Final log at the end of epoch
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": total_loss / len(train_loader),
            "val_loss": val_loss,
            "val_mAP@0.5": mean_ap,
            **category_ap
        })


In [ ]:
wandb.agent(sweep_id, function=wandb_train, count=6)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: t5ku7e85 with config:
wandb: 	batch_size: 2
wandb: 	learning_rate: 1e-05
wandb: 	weight_decay: 0.0001


Exception in thread Thread-26 (_run_job):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "<ipython-input-24-c33ffec1190c>", line 9, in wandb_train
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_init.py", line 1544, in init
    wandb._sentry.reraise(e)
  File "/usr/local/lib/python3.11/dist-packages/wandb/analytics/sentry.py", line 156, in reraise
    raise exc.with_traceback(sys.exc_info()[2])
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_init.py", line 1530, in init
    return wi.init(run_settings, run_config)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_init.py", line 799, in init
    wandb.run.finish()
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_run.py", line 387, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fi

Exception in thread Thread-27 (_run_job):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "<ipython-input-24-c33ffec1190c>", line 9, in wandb_train
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_init.py", line 1544, in init
    wandb._sentry.reraise(e)
  File "/usr/local/lib/python3.11/dist-packages/wandb/analytics/sentry.py", line 156, in reraise
    raise exc.with_traceback(sys.exc_info()[2])
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_init.py", line 1530, in init
    return wi.init(run_settings, run_config)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_init.py", line 799, in init
    wandb.run.finish()
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_run.py", line 387, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fi

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x79f80538f090>> (for post_run_cell):


^

BrokenPipeError: [Errno 32] Broken pipe